In [1]:
import sys 
sys.path.append("..") 
import graphgallery as gg
from graphgallery import functional as gf
from graphgallery.datasets import Planetoid, NPZDataset
import datareader, sharedutils
import torch
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils import data as Data
from model_lif_fc_with_val import model_lif_fc
print('loaded')

data = Planetoid('cora', root="~/datasets/datafromgg", verbose=False)

graph = data.graph
splits = data.split_nodes(random_state=15)

################### Attacker model ############################
attacker = gg.attack.untargeted.NodeEmbeddingAttack(graph, seed=42).process()
attacker.attack(1000, K=None)

################### Victim model ############################
# Before attack
trainer = gg.gallery.nodeclas.GCN(seed=123).make_data(graph).build()
his = trainer.fit(splits.train_nodes,
                  splits.val_nodes,
                  verbose=1,
                  epochs=100)
original_result = trainer.evaluate(splits.test_nodes)

# After attack
trainer = gg.gallery.nodeclas.GCN(seed=123).make_data(attacker.g).build()
his = trainer.fit(splits.train_nodes,
                  splits.val_nodes,
                  verbose=1,
                  epochs=100)
perturbed_result = trainer.evaluate(splits.test_nodes)

loaded
Training...
100/100 [====================] - Total: 1.11s - 11ms/step - loss: 0.167 - accuracy: 0.979 - val_loss: 0.785 - val_accuracy: 0.752
Testing...
1/1 [====================] - Total: 54.90ms - 54ms/step - loss: 0.777 - accuracy: 0.805
Training...
100/100 [====================] - Total: 903.03ms - 9ms/step - loss: 0.179 - accuracy: 0.993 - val_loss: 0.853 - val_accuracy: 0.752
Testing...
1/1 [====================] - Total: 54.07ms - 54ms/step - loss: 0.801 - accuracy: 0.762


In [2]:
# #Using spiking
dataname = "cora"
conf, cnf = sharedutils.read_config(), {}
rd = datareader.ReadData("~/datasets/datafromgg")

# # get fixed split

mat, tag = rd.attacked_conv_graph(attacker.g)
# mat, tag = rd.conv_graph(data_fixed)
tr_ind, val_ind, ts_ind = data.split_nodes().train_nodes, data.split_nodes().val_nodes, data.split_nodes().test_nodes
tr_mat = mat[tr_ind]
val_mat = mat[val_ind]
ts_mat = mat[ts_ind]
tr_tag = tag[tr_ind]
val_tag = tag[val_ind]
ts_tag = tag[ts_ind]


#get random split
# data = rd.read_raw_data(dataname)
# mat, tag = rd.conv_graph(data)
# tr_val_mat, ts_mat, tr_val_tag, ts_tag = rd.get_random_splited_data(mat, tag, 0.2)
k = pd.DataFrame(mat)
u = k.describe()
# print("tr_mat.shape()",u)

cnf.update(conf['shared_conf'])
cnf.update(conf['snn'][dataname])
# may be some new params
print("cnf:",cnf["batch_size"])
cnf['log_dir'] = conf['snn']['log_dir']
if cnf['v_reset'] == -100: cnf['v_reset'] = None
train_data_loader, val_data_loader, test_data_loader = rd.tr_ts_val_numpy2dataloader(tr_mat, ts_mat, val_mat, tr_tag, ts_tag, val_tag,batch_size=cnf["batch_size"] )
print("train, valiadation,test's len:", len(train_data_loader), len(val_data_loader), len(test_data_loader))
n_nodes, n_feat, n_flat = mat.shape[0], mat.shape[1], 1
print("data: %s, num_node_classes: %d" % (dataname, data.graph.num_classes))
print(cnf)
ret = model_lif_fc(device=cnf["device"], dataset_dir=cnf["dataset_dir"],
                 dataname=dataname, batch_size=cnf["batch_size"], 
                 learning_rate=cnf["learning_rate"], T=cnf["T"], tau=cnf["tau"], 
                 v_reset=cnf["v_reset"], v_threshold=cnf["v_threshold"],
                 train_epoch=cnf["train_epoch"], log_dir=cnf["log_dir"], n_labels=data.graph.num_classes,
                 n_dim0=n_nodes, n_dim1=n_flat, n_dim2=n_feat, train_data_loader=train_data_loader,
                 val_data_loader=val_data_loader, test_data_loader=test_data_loader)

################### Results ############################
print(f"original prediction {original_result.accuracy:.2%}")
print(f"perturbed prediction {perturbed_result.accuracy:.2%}")
print(
    f"The accuracy has gone down {original_result.accuracy-perturbed_result.accuracy:.2%}"
)

print(f"original prediction {ret:.2%}")



cnf: 64
train, valiadation,test's len: 3 8 16
data: cora, num_node_classes: 7
{'device': 'cuda:1', 'dataset_dir': 'graphfromgg', 'test_size': 0.2, 'batch_size': 64, 'learning_rate': 0.02, 'T': 300, 'tau': 50, 'v_reset': None, 'v_threshold': 0.2, 'train_epoch': 200, 'ggp_embedding': False, 'num_per_class': 20, 'log_dir': '/home/zlzhu/snn/bsgcn/handcode/tmpdir/snn'}
testset'acc: device=cuda:1, dataset=cora, batch_size=64, learning_rate=0.02, T=300, max_test_accuracy=0.7630, sops_per_nodes:  4186.0000, num_s1: 5142, num_s2: 212, num_s_per_node: 5354y=0.7360, train_times=600
original prediction 80.50%
perturbed prediction 76.20%
The accuracy has gone down 4.30%
original prediction 76.30%
